In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv', index_col = 'id')
oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
holidays = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv', index_col = 'id')
submission = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')

In [ ]:
y_train = train['sales']
train.drop(columns = ['sales'] , inplace = True , axis = 1)
X_train = train
X_test = test

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
X_train['days']  = (pd.to_datetime(X_train['date']) - pd.Timestamp('20130101')).dt.days
X_train.drop(columns = ['date'] , inplace = True , axis = 1)
X_train.head()

In [ ]:
X_test['days']  = (pd.to_datetime(X_test['date']) - pd.Timestamp('20130101')).dt.days
X_test.drop(columns = ['date'] , inplace = True , axis = 1)
X_test.head()

In [ ]:
stores.head()

In [ ]:
oil['days']  = (pd.to_datetime(oil['date']) - pd.Timestamp('20130101')).dt.days
oil.drop(columns = ['date'] , inplace = True , axis = 1)
oil = oil.dropna()
oil.head()

In [ ]:
new_column = [0 for i in range(3000888)]
keys = oil['days'].values
values = oil['dcoilwtico'].values
dictionary = dict(zip(keys, values))
for i in range (3000888):
    ind = X_train['days'].values[i]
    try:
        value = dictionary[ind]
        new_column[i] = value
    except:
        pass

X_train['oil'] = new_column

new_column = ['none' for i in range(3000888)]
keys = stores['store_nbr'].values
values = stores['city'].values
dictionary = dict(zip(keys, values))
for i in range (3000888):
    ind = X_train['store_nbr'].values[i]
    try:
        value = dictionary[ind]
        new_column[i] = value
    except:
        pass

X_train['city'] = new_column

X_train.head

In [ ]:
new_column = [0 for i in range(28512)]
keys = oil['days'].values
values = oil['dcoilwtico'].values
dictionary = dict(zip(keys, values))
for i in range (28512):
    ind = X_test['days'].values[i]
    try:
        value = dictionary[ind]
        new_column[i] = value
    except:
        pass

X_test['oil'] = new_column

new_column = ['none' for i in range(28512)]
keys = stores['store_nbr'].values
values = stores['city'].values
dictionary = dict(zip(keys, values))
for i in range (28512):
    ind = X_test['store_nbr'].values[i]
    try:
        value = dictionary[ind]
        new_column[i] = value
    except:
        pass

X_test['city'] = new_column

X_test.head

In [ ]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
model = LinearRegression()
def k_fold(X, y, n_splits = 5):
    step = len(y) // n_splits
    accuracies = []
    for n in range(1, n_splits):
        X_train = X[0 : step * n]
        X_test = X[step * n: step * (n + 1)]
        y_train = y[0 : step * n]
        y_test = y[step * n: step * (n + 1)]
        
        model.fit(X_train , y_train)
        
        y_pred = [max(item, 0) for item in model.predict(X_test)]
        accuracies.append(sqrt(mean_squared_error(list(y_pred), list(y_test))))
    return accuracies
lst = k_fold(X_train, y_train)
print(sum(lst) / len(lst))

In [ ]:

model = LinearRegression()
model.fit(X_train, y_train)
pred  = model.predict(X_test)
pred

In [ ]:
submission['sales'] = pred
submission.to_csv('submission.csv' , index = False)